# Fourieranalyse und Akustik
von Fabian Wolter uns Selin Kabak
### 1. Bestimmung der Tonhöhe eines akustischen Signals

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

csv_file = "AAAAA.csv"
data = np.genfromtxt(
    csv_file,
    delimiter=';',
    skip_header=1,
    usecols=[1],
    # replace method needs a String, so convert bytes to string
    converters={1: lambda s: float(s.decode('utf-8').replace(',', '.'))},
)
time = np.genfromtxt(
    csv_file,
    delimiter=';',
    skip_header=1,
    usecols=[0],
    converters={0: lambda s: float(s.decode('utf-8').replace(',', '.'))},
)

plt.plot(time, data)
plt.xlabel('Zeit in ms')
plt.ylabel('Amplitude')
plt.title('Signal')
plt.show()

# kann man noch ablesen
print("Signaldauer beträgt:", time[-1], "ms,", time[-1] / 1000, "s")

# Anzahl Werte in der CSV Datei
print("Signallänge beträgt:", len(time), "Werte")

# Signallänge / Signaldauer = Abtastrate
print("Abtastrate:", len(time), "/", time[-1] / 1000, "s =", len(time) / (time[-1] / 1000), "Hz")

# Abtastintervall = 1 / Abtastrate
print("Abtastintervall: 1 /", len(time) / (time[-1] / 1000), "s,", len(time) / (time[-1] / 1000) * 1000, "ms")

# Grundperiode = Signaldauer / Anzahl Perioden (kann man abzählen)
print("Grundperiode:", time[-1], "/ 14 =", time[-1] / 14)

# Grundfrequenz = 1 / Grundperiode
print("Grundfrequenz:", 14 / (time[-1] / 1000), "Hz")

In [ ]:
# Zeit zwischen zwei aufeinanderfolgenden Abtastpunkten
T = (time[-1] / 1000) / len(time)

# fft zerlegt das Signal in seine Frequenzkomponenten
ft = np.fft.fft(data)

# fftfreq gibt die Frequenzen der einzelnen Komponenten zurück, T ist das Abtastintervall
freq = np.fft.fftfreq(len(ft), d=T)

# es gibt einen Real- und Imaginärteil, weil die Fouriertransformation 
# als Summe komplexer Exponentialfunktionen dargestellt wird
plt.plot(freq, ft.real, freq, ft.imag)
plt.xlabel('Frequenz in Hz')
plt.ylabel('Amplitude')
plt.title('Fouriertransformation')
plt.show()

plt.plot(freq, ft.real, freq, ft.imag)
plt.xlim(-700, 700)
plt.xlabel('Frequenz in Hz')
plt.ylabel('Amplitude')
plt.title('Fouriertransformation ZOOM')
plt.show()

max_freq = freq[np.argmax(ft.imag)]
print("Die berechnete Frequenz beträgt:", max_freq, "Hz")
max_amplitude = np.max(ft.imag)
print("mit einer Amplitude von:", max_amplitude)

### 2. Frequenzgang von Lautsprechern

In [ ]:
# load data
csv_file = "big.csv"
freqs_big = np.genfromtxt(
    csv_file,
    delimiter=',',
    skip_header=1,
    usecols=[0],
)
phase_big = np.genfromtxt(
    csv_file,
    delimiter=',',
    skip_header=1,
    usecols=[1],
)
amplitudes_big = np.genfromtxt(
    csv_file,
    delimiter=',',
    skip_header=1,
    usecols=[2],
)
csv_file = "small.csv"
freqs_small = np.genfromtxt(
    csv_file,
    delimiter=',',
    skip_header=1,
    usecols=[0],
)
phase_small = np.genfromtxt(
    csv_file,
    delimiter=',',
    skip_header=1,
    usecols=[1],
)
amplitudes_small = np.genfromtxt(
    csv_file,
    delimiter=',',
    skip_header=1,
    usecols=[2],
)

# Amplitudengang von Großem Lautsprecher:
# Spitze bei 100 Hz, sinkt danach kleinere Spitzen
a_ref = amplitudes_big[0]
for i in range(len(amplitudes_big)):
    # Amplitude in dB umrechnen. Formel: 20 * log10(amplitude / referenzamplitude)
    amplitudes_big[i] = 20 * np.log10(amplitudes_big[i] / a_ref)
    
# Amplitudengang von Kleinem Lautsprecher:
# Große Spitze bei 1000 Hz ca., danach Abfall mit kleineren Schwankungen
a_ref = amplitudes_small[0]
for i in range(len(amplitudes_small)):
    amplitudes_small[i] = 20 * np.log10(amplitudes_small[i] / a_ref)

# Phasengang von Großem Lautsprecher
for i in range(len(phase_big)):
    # Phasenwinkel in Grad umrechnen. Formel: -(phasenwinkel / 1000) * frequenz * 360
    phase_big[i] = -(phase_big[i] / 1000) * freqs_big[i] * 360  
    
# Phasengang von Kleinem Lautsprecher
for i in range(len(phase_small)):
    phase_small[i] = -(phase_small[i] / 1000) * freqs_small[i] * 360

plt.subplot(2, 1, 1)
plt.semilogx(freqs_big, amplitudes_big)
plt.xlabel('Frequenz in Hz')
plt.ylabel('Amplitude in dB')
plt.title('Bode Diagramm großer Lautsprecher')
plt.subplot(2, 1, 2)
plt.semilogx(freqs_big, phase_big)
plt.xlabel('Frequenz in Hz')
plt.ylabel('Phasenwinkel')
plt.show()

plt.subplot(2, 1, 1)
plt.semilogx(freqs_small, amplitudes_small)
plt.xlabel('Frequenz in Hz')
plt.ylabel('Amplitude in dB')
plt.title('Bode Diagramm kleiner Lautsprecher')
plt.subplot(2, 1, 2)
plt.semilogx(freqs_small, phase_small)
plt.xlabel('Frequenz in Hz')
plt.ylabel('Phasenwinkel')
plt.show()

# Welcher Frequenzbereich ist sinnvoll zu messen?
# Laut dem Datenblatt: 70 Hz bis 13 kHz
# Frequenzen außerhalb können ungenau sein
# Beide Lautsprecher haben eine ähnliche Charakteristik im selben Frequenzbereich
# Großer Lautsprecher war etwas besser in tieferen Frequenzen
# Kleiner Lautsprecher war etwas besser in mittleren Frequenzen